In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

# Chargement des données
train_home = pd.read_csv('Datas/X_Train_Data/train_home_team_statistics_df.csv')
train_away = pd.read_csv('Datas/X_Train_Data/train_away_team_statistics_df.csv')
y_train = pd.read_csv('Datas/Y_train_1rknArQ.csv')

test_home = pd.read_csv('Datas/X_Test_Data/test_home_team_statistics_df.csv')
test_away = pd.read_csv('Datas/X_Test_Data/test_away_team_statistics_df.csv')
y_test = pd.read_csv('Datas/Y_test_random_sEE2QeA.csv')

# Nettoyage des colonnes non numériques
train_home_clean = train_home.drop(columns=["LEAGUE", "TEAM_NAME"])
train_away_clean = train_away.drop(columns=["LEAGUE", "TEAM_NAME"])


# Préfixe pour éviter conflits de noms
train_home_clean.columns = ["HOME_" + col for col in train_home_clean.columns]
train_away_clean.columns = ["AWAY_" + col for col in train_away_clean.columns]

test_home.columns = ["HOME_" + col for col in test_home.columns]
test_away.columns = ["AWAY_" + col for col in test_away.columns]

# Fusion des données
X_train = pd.concat([train_home_clean, train_away_clean], axis=1)
X_test = pd.concat([test_home, test_away], axis=1)

# Remplissage des NaN
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Construction de la cible à partir de Y_train
ids = y_train["ID"]
ids_test = y_test["ID"]

# Convertir en labels : 0 = away, 1 = draw, 2 = home
y_labels = y_train[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

y_test_labels = y_test[["HOME_WINS", "DRAW", "AWAY_WINS"]].idxmax(axis=1).map({
    "AWAY_WINS": 0,
    "DRAW": 1,
    "HOME_WINS": 2
})

# Définition du modèle RandomForest
model = RandomForestClassifier(random_state=42)

# Définition des hyperparamètres à tester avec GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres dans la forêt
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],  # Nombre minimum d’échantillons pour diviser un noeud
    'min_samples_leaf': [1, 2, 4],  # Nombre minimum d’échantillons par feuille
    'max_features': ['sqrt', 'log2', None],  # Nombre maximum de features pour chaque split
    'bootstrap': [True, False],  # Utilisation de l’échantillonnage avec remise ?
    'class_weight': ['balanced', None],  # Pondération des classes
}

# Utilisation de GridSearchCV pour optimiser les hyperparamètres
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Entraînement avec GridSearchCV
grid_search.fit(X_train, y_labels)

# Affichage des meilleurs paramètres
print("Meilleurs paramètres trouvés :")
print(grid_search.best_params_)

# Meilleur modèle trouvé
best_model = grid_search.best_estimator_

# Faire des prédictions avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Affichage des résultats
print("Rapport de classification avec le meilleur modèle :")
print(classification_report(y_test_labels, y_pred, target_names=["AWAY_WINS", "DRAW", "HOME_WINS"]))

# Enregistrement du modèle optimisé
import joblib
joblib.dump(best_model, 'random_forest_optimise.pkl')



KeyError: "['LEAGUE', 'TEAM_NAME'] not found in axis"